In [1]:
import pandas as pd
import numpy as np
import requests as rq
import time
from requests.exceptions import HTTPError
import json
from jsonschema import validate
from riotwatcher import LolWatcher, ApiError

In [2]:
API_KEY = "RGAPI-e846276f-d79b-4e8a-9777-5bb773957654"
base_diamond = "https://br1.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/"
base_challenger = "https://br1.api.riotgames.com/lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5/"
base_master = "https://br1.api.riotgames.com/lol/league/v4/masterleagues/by-queue/RANKED_SOLO_5x5"
base_grand_master = "https://br1.api.riotgames.com/lol/league/v4/grandmasterleagues/by-queue/RANKED_SOLO_5x5"

base_summoner_infos = "https://br1.api.riotgames.com/lol/summoner/v4/summoners/by-name/"

In [3]:
watcher = LolWatcher(API_KEY)
my_region = 'br1'

In [4]:
def makeRequest(string):
    try:
        response = rq.get(string)
        response.raise_for_status()
    except HTTPError:
        if response.status_code == 429:
            time.sleep(120)
            makeRequest(string)
    return response.json()

In [5]:
def getLowEloInfos(json):
    df = pd.DataFrame(json)
    df["tier"] = df['tier'] + df["rank"]
    df.drop(["leagueId","queueType","rank","miniSeries"], axis=1,inplace=True)
    return df   

In [6]:
def getHighEloInfos(json):
    df_aux = pd.DataFrame(json["entries"])
    df_aux.drop("rank", axis=1, inplace=True)
    df_aux["tier"] = "MASTER"
    df_aux = df_aux.reindex(
        columns=['tier', 'summonerId', 'summonerName', 
                 'leaguePoints', 'wins', 'losses','veteran',
                 'inactive', 'freshBlood', 'hotStreak'])
    return df_aux

In [16]:
%%time
divisions = ["I","II","III","IV"]
tiers = ["DIAMOND","MASTER","GRANDMASTER","CHALLENGER"]
df = pd.DataFrame()
for i in tiers:
    if i == "DIAMOND":
        for j in divisions:
            string = base_diamond + i + "/" + j + "?api_key=" + API_KEY
            df_aux = getLowEloInfos(makeRequest(string))
            df = df.append(df_aux,ignore_index = True)
    
    elif i == "MASTER":
        string = base_master + "?api_key=" + API_KEY
        df_aux = getHighEloInfos(makeRequest(string))
        df = df.append(df_aux,ignore_index = True)
    
    
    elif i == "GRANDMASTER":
        string = base_grand_master + "?api_key=" + API_KEY
        df_aux = getHighEloInfos(makeRequest(string))
        df = df.append(df_aux,ignore_index = True)
    
    else:
        string = base_challenger + "?api_key=" + API_KEY
        df_aux = getHighEloInfos(makeRequest(string))
        df = df.append(df_aux,ignore_index = True)
        

CPU times: user 278 ms, sys: 33.5 ms, total: 312 ms
Wall time: 6.24 s


In [17]:
df.head()

,tier,summonerId,summonerName,leaguePoints,wins,losses,veteran,inactive,freshBlood,hotStreak
0,DIAMONDI,a6XNnr-OC1673HAJaeCorjI1T9Sbe7gYJIBpbsu_CllJIOI,LokiFc,100,188,177,False,False,False,False
1,DIAMONDI,6YbR4YdY-AewAr2ihQtT_qHhJyNGtHUZNDLexZBQGU7ZZGQ,Janna Jana Janna,6,291,291,False,False,False,False
2,DIAMONDI,h_TlijF-s3RAtZaCyUVvBvgO-GGVuG60TqH3h7cTGrOKT18,lepanta,75,120,100,False,False,False,False
3,DIAMONDI,8fb8V6VEtBGfFZgq3Fyv3eEJ43UQyIeFOCm0SKEQvgu673M,i still hate you,0,350,298,False,False,False,False
4,DIAMONDI,yMjxJWZ9bgYm-QRQTLetKFsSDxM04GaLxNSdQSOpP_NsBW0,catch a fireee,0,423,389,True,False,False,False


In [18]:
df.to_csv("usersBasicInfos.csv",index=False)

In [7]:
df = pd.read_csv("usersBasicInfos.csv",index_col="summonerName")

In [8]:
df.shape

(2681, 9)

In [15]:
df.head()

,tier,summonerId,leaguePoints,wins,losses,veteran,inactive,freshBlood,hotStreak
summonerName,,,,,,,,,
LokiFc,DIAMONDI,a6XNnr-OC1673HAJaeCorjI1T9Sbe7gYJIBpbsu_CllJIOI,100,188,177,False,False,False,False
Janna Jana Janna,DIAMONDI,6YbR4YdY-AewAr2ihQtT_qHhJyNGtHUZNDLexZBQGU7ZZGQ,6,291,291,False,False,False,False
lepanta,DIAMONDI,h_TlijF-s3RAtZaCyUVvBvgO-GGVuG60TqH3h7cTGrOKT18,75,120,100,False,False,False,False
i still hate you,DIAMONDI,8fb8V6VEtBGfFZgq3Fyv3eEJ43UQyIeFOCm0SKEQvgu673M,0,350,298,False,False,False,False
catch a fireee,DIAMONDI,yMjxJWZ9bgYm-QRQTLetKFsSDxM04GaLxNSdQSOpP_NsBW0,0,423,389,True,False,False,False


#### Agora vamos enriquecer esses dados com mais informações 

In [9]:
def checkFields(response, fields):
    for i in fields:
        if i not in response:
            return False
    return True

In [10]:
def getSummonerInfos(summonerNames):
    list_dicts = []
    count = 0
    for i in summonerNames:
        count+=1
        if (count % 100) == 0:
            df_temp = pd.DataFrame(list_dicts)
            df_temp.to_csv("DataSummornerTemp.csv")
        string = base_summoner_infos + i + "?api_key=" + API_KEY
        response = makeRequest(string)
        infos_dict = {}
        if checkFields(response,["accountId","summonerLevel","puuid"]):
            infos_dict["summonerName"] = i
            infos_dict["accountId"] = response["accountId"]
            infos_dict["summonerLevel"] = response["summonerLevel"]
            infos_dict["puuid"] = response["puuid"]
            list_dicts.append(infos_dict)
    return list_dicts

In [40]:
names = df.summonerName.to_list()

In [31]:
len(names)

2681

In [11]:
df_backup = pd.read_csv("DataSummornerTemp.csv")

In [53]:
%%time
infos_plus = getSummonerInfos(names)

CPU times: user 1min 27s, sys: 7.83 s, total: 1min 35s
Wall time: 1h 24min 35s


In [55]:
len(infos_plus)

2644

In [56]:
df_summoner = pd.DataFrame(infos_plus)

In [58]:
df_summoner.head()

,summonerName,accountId,summonerLevel,puuid
0,LokiFc,bk1m2_GS-2L9loYOKil6EbsEHnQOCY15LRVByLuQvjf46Cc,289,AvAT9pAFILqL0Eo0kWmFOXug4KyIBh4QprnfHU4Ao2e3Bj...
1,Janna Jana Janna,0-0irM82Yyb9ls9AQ2PGuGHjqn2pWBFVAuKEjM4qXeu5zHk,329,xo9H9-nK8p0qks6P39ceIIluPBqAGujYR1YZSC_JqWZxYD...
2,lepanta,B6nuFHgma2xw3B4ZYsSQeb38pgQxgajWykeB8OEZHm0sSO8,130,z2NP0f9IPVNevXHBEiZ5wjOHHwmqifJM0St3OpTjgI8xon...
3,i still hate you,4FnPhlgfii3JW283EbPr2Zl4ZPYoVdk6Qfjc7LIPP8MrZEI,224,CZq0S_gurM-JdaLAI8mZxCHhvxWGSen9ipXUt6QCOkyNDz...
4,catch a fireee,p5gtfCphMzSZUHv_hGKI4gAdej_ezQcgvHelm-rvB_878ec,131,EVZfgcJKxbwTCbnvj72it5d9gBeI52DNAoOL7dDKbWgwDp...


In [59]:
df_summoner.to_csv("summonerInfosPlus.csv",index=False)

In [12]:
df_summoner = pd.read_csv("summonerInfosPlus.csv",index_col="summonerName")

In [13]:
df.head(2)


,tier,summonerId,leaguePoints,wins,losses,veteran,inactive,freshBlood,hotStreak
summonerName,,,,,,,,,
LokiFc,DIAMONDI,a6XNnr-OC1673HAJaeCorjI1T9Sbe7gYJIBpbsu_CllJIOI,100,188,177,False,False,False,False
Janna Jana Janna,DIAMONDI,6YbR4YdY-AewAr2ihQtT_qHhJyNGtHUZNDLexZBQGU7ZZGQ,6,291,291,False,False,False,False


In [14]:
df_summoner.head(2)

,accountId,summonerLevel,puuid
summonerName,,,
LokiFc,bk1m2_GS-2L9loYOKil6EbsEHnQOCY15LRVByLuQvjf46Cc,289,AvAT9pAFILqL0Eo0kWmFOXug4KyIBh4QprnfHU4Ao2e3Bj...
Janna Jana Janna,0-0irM82Yyb9ls9AQ2PGuGHjqn2pWBFVAuKEjM4qXeu5zHk,329,xo9H9-nK8p0qks6P39ceIIluPBqAGujYR1YZSC_JqWZxYD...


In [15]:
df_summonerFinal = df.join(df_summoner,how="inner",on="summonerName")

In [16]:
df_summonerFinal.columns

Index(['tier', 'summonerId', 'leaguePoints', 'wins', 'losses', 'veteran',
       'inactive', 'freshBlood', 'hotStreak', 'accountId', 'summonerLevel',
       'puuid'],
      dtype='object')

In [17]:
df_summonerFinal.to_csv("SummonerFinalData.csv")

In [18]:
df_summonerFinal.head()

,tier,summonerId,leaguePoints,wins,losses,veteran,inactive,freshBlood,hotStreak,accountId,summonerLevel,puuid
summonerName,,,,,,,,,,,,
LokiFc,DIAMONDI,a6XNnr-OC1673HAJaeCorjI1T9Sbe7gYJIBpbsu_CllJIOI,100,188,177,False,False,False,False,bk1m2_GS-2L9loYOKil6EbsEHnQOCY15LRVByLuQvjf46Cc,289,AvAT9pAFILqL0Eo0kWmFOXug4KyIBh4QprnfHU4Ao2e3Bj...
Janna Jana Janna,DIAMONDI,6YbR4YdY-AewAr2ihQtT_qHhJyNGtHUZNDLexZBQGU7ZZGQ,6,291,291,False,False,False,False,0-0irM82Yyb9ls9AQ2PGuGHjqn2pWBFVAuKEjM4qXeu5zHk,329,xo9H9-nK8p0qks6P39ceIIluPBqAGujYR1YZSC_JqWZxYD...
lepanta,DIAMONDI,h_TlijF-s3RAtZaCyUVvBvgO-GGVuG60TqH3h7cTGrOKT18,75,120,100,False,False,False,False,B6nuFHgma2xw3B4ZYsSQeb38pgQxgajWykeB8OEZHm0sSO8,130,z2NP0f9IPVNevXHBEiZ5wjOHHwmqifJM0St3OpTjgI8xon...
i still hate you,DIAMONDI,8fb8V6VEtBGfFZgq3Fyv3eEJ43UQyIeFOCm0SKEQvgu673M,0,350,298,False,False,False,False,4FnPhlgfii3JW283EbPr2Zl4ZPYoVdk6Qfjc7LIPP8MrZEI,224,CZq0S_gurM-JdaLAI8mZxCHhvxWGSen9ipXUt6QCOkyNDz...
catch a fireee,DIAMONDI,yMjxJWZ9bgYm-QRQTLetKFsSDxM04GaLxNSdQSOpP_NsBW0,0,423,389,True,False,False,False,p5gtfCphMzSZUHv_hGKI4gAdej_ezQcgvHelm-rvB_878ec,131,EVZfgcJKxbwTCbnvj72it5d9gBeI52DNAoOL7dDKbWgwDp...


   ### Matchs Players

In [19]:
df_summoners = pd.read_csv("SummonerFinalData.csv")

In [20]:
summoners_id = df_summoners.accountId.to_list()

In [21]:
len(summoners_id)

2644

In [32]:
def getSummonerMatches(accountId):
    result = watcher.match.matchlist_by_account(my_region,accountId)["matches"]
    return result
    

In [36]:
summoner_list = []
for summoner in summoners_id:
    matches = {}
    matches["accountId"] = summoner
    try:
        matches["listMatches"] = getSummonerMatches(summoner)
    except:
        time.sleep(5)
        matches["listMatches"] = getSummonerMatches(summoner)     
    summoner_list.append(matches)  

In [37]:
df_matches = pd.DataFrame(summoner_list)

In [38]:
df_matches.to_csv("matchesBasicInfos.csv")

In [97]:
df_all = []

In [102]:
for i in df_matches.values:
    accountId = i[0]
    df_temp = pd.DataFrame(i[1])
    df_temp["accountId"] = accountId
    df_all.append(df_temp)

In [103]:
df_final = pd.concat(df_all)


In [105]:
df_final.to_csv("matchByAccounbtId.csv",index=None)

 ### Matche Stats

In [25]:
matchesbySummoner = []
for account in summoners_id[0:5]:
    matches = getSummonerMatches(account)
    matches_list = matches['matches']
    for game in total_matches:
        match_detail = watcher.match.by_id(my_region, game['gameId'])
        participants = []
        for row in match_detail['participants']:
            participants_row = {}
            participants_row = game['gameId']
            participants_row['champion'] = row['championId']
            participants_row['spell1'] = row['spell1Id']
            participants_row['spell2'] = row['spell2Id']
            participants_row['win'] = row['stats']['win']
            participants_row['kills'] = row['stats']['kills']
            participants_row['deaths'] = row['stats']['deaths']
            participants_row['assists'] = row['stats']['assists']
            participants_row['totalDamageDealt'] = row['stats']['totalDamageDealt']
            participants_row['goldEarned'] = row['stats']['goldEarned']
            participants_row['champLevel'] = row['stats']['champLevel']
            participants_row['totalMinionsKilled'] = row['stats']['totalMinionsKilled']
            participants_row['item0'] = row['stats']['item0']
            participants_row['item1'] = row['stats']['item1']
            participants.append(participants_row)
        


In [47]:
my_matches["matches"]

[{'platformId': 'BR1',
  'gameId': 2056742288,
  'champion': 74,
  'queue': 420,
  'season': 13,
  'timestamp': 1599723309763,
  'role': 'SOLO',
  'lane': 'TOP'},
 {'platformId': 'BR1',
  'gameId': 2056692263,
  'champion': 101,
  'queue': 420,
  'season': 13,
  'timestamp': 1599720773237,
  'role': 'SOLO',
  'lane': 'MID'},
 {'platformId': 'BR1',
  'gameId': 2056740267,
  'champion': 26,
  'queue': 420,
  'season': 13,
  'timestamp': 1599717862468,
  'role': 'SOLO',
  'lane': 'TOP'},
 {'platformId': 'BR1',
  'gameId': 2056648824,
  'champion': 34,
  'queue': 420,
  'season': 13,
  'timestamp': 1599714908565,
  'role': 'SOLO',
  'lane': 'MID'},
 {'platformId': 'BR1',
  'gameId': 2056656908,
  'champion': 101,
  'queue': 420,
  'season': 13,
  'timestamp': 1599712727798,
  'role': 'SOLO',
  'lane': 'MID'},
 {'platformId': 'BR1',
  'gameId': 2056549341,
  'champion': 26,
  'queue': 440,
  'season': 13,
  'timestamp': 1599705990164,
  'role': 'SOLO',
  'lane': 'TOP'},
 {'platformId': 'BR1